In [3]:
import numpy as np
from scipy.sparse import csr_matrix, dok_matrix
import scipy.sparse
from scipy.sparse import linalg as slinalg

adjacency_path = "./data/movielens20M.train.npz"
adjacency_train = scipy.sparse.load_npz(adjacency_path)

In [4]:
def get_pmi_matrix(adjacency, cds, neg, pos):
    sum_w = np.array(adjacency.sum(axis=1))[:, 0]
    sum_c = np.array(adjacency.sum(axis=0))[0, :]
    if cds != 1:
        sum_c = sum_c ** cds
    sum_total = sum_c.sum()
    sum_w = np.reciprocal(sum_w)
    sum_c = np.reciprocal(sum_c)

    pmi = multiply_by_rows(adjacency, sum_w)
    pmi = multiply_by_columns(pmi, sum_c)
    pmi = pmi * sum_total
    pmi.data = np.log(pmi.data) - np.log(neg) + np.log(pos)
    return pmi

def multiply_by_rows(matrix, row_coefs):
    normalizer = dok_matrix((len(row_coefs), len(row_coefs)))
    normalizer.setdiag(row_coefs)
    return normalizer.tocsr().dot(matrix)


def multiply_by_columns(matrix, col_coefs):
    normalizer = dok_matrix((len(col_coefs), len(col_coefs)))
    normalizer.setdiag(col_coefs)
    return matrix.dot(normalizer.tocsr())

cds = 1.0
neg = 1
pos = 1
pmi = get_pmi_matrix(adjacency_train, cds, neg, pos)

In [5]:
pmi.data.min()

-2.872420120645959

In [6]:
dim = 100
#U, s, Vt = slinalg.svds(pmi, dim)
U, s, Vt = slinalg.svds(adjacency_train, dim)

In [7]:
#np.save("./data/movielens_users_pos=100", U)
#np.save("./data/movielens_movies_pos=100", Vt.T)
#np.save("./data/movielens_singular_vals_pos=100", s)

np.save("./data/movielens20M_users_svd", U)
np.save("./data/movielens20M_movies_svd", Vt.T)
np.save("./data/movielens20M_singular_vals_svd", s)

In [13]:
scipy.sparse.save_npz("./data/movielens.adjacency.train.npz", adjacency_train)
scipy.sparse.save_npz("./data/movielens.adjacency.test.npz", adjacency_test)